https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?resource=download

In [38]:
from sklearn.model_selection import train_test_split
import os
import torch
import sys
from pathlib import Path

In [39]:
sys.path.append(os.path.abspath("..")) 

In [40]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    
if not (project_root / 'src').exists():
    print(f"⚠️ Ostrzeżenie: Nie znaleziono katalogu 'src' w {project_root}")
    
print(f"Dodano do ścieżki: {project_root}")

Dodano do ścieżki: c:\Users\bartekb\Desktop\inzynierka\fromscratch-transformer-classifier


In [41]:
import pandas as pd
from src.textclf_transformer import * 

In [42]:
from datasets import load_dataset, concatenate_datasets

ds = load_dataset("imdb")

merged = concatenate_datasets([ds["test"], ds["train"]])
df = pd.DataFrame(merged)

In [43]:
df['text']

0        I love sci-fi and am willing to put up with a ...
1        Worth the entertainment value of a rental, esp...
2        its a totally average film with a few semi-alr...
3        STAR RATING: ***** Saturday Night **** Friday ...
4        First off let me say, If you haven't enjoyed a...
                               ...                        
49995    A hit at the time but now better categorised a...
49996    I love this movie like no other. Another time ...
49997    This film and it's sequel Barry Mckenzie holds...
49998    'The Adventures Of Barry McKenzie' started lif...
49999    The story centers around Barry McKenzie who mu...
Name: text, Length: 50000, dtype: object

In [44]:
df.head()

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0


In [45]:
df["text"] = df["text"].str.replace(r"<br\s*/?>", "", regex=True)

In [46]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42, stratify=test_df['label'])

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

Train: 40000 | Val: 5000 | Test: 5000


In [47]:
tok = WordPieceTokenizerWrapper()

In [48]:
tok.load(tokenizer_dir=r"..\..\src\textclf_transformer\tokenizer\IMDB_tokenizer")

In [49]:
tokens_train = tok.encode_pandas(train_df, 'text',
                                  max_length=512, 
                                  label_col='label'
                                 )
tokens_val = tok.encode_pandas(val_df, 'text',
                                max_length=512,
                                label_col='label')
tokens_test = tok.encode_pandas(test_df, 'text',
                                 max_length=512,
                                 label_col='label')

In [50]:
print(tok.tokenizer.decode(tokens_train[0][0]))
print(tokens_train[0][0])  # label

[CLS] i ' d love to give kolchak a higher rating but the show quickly went from scary / suspenseful to silly. abc ' s fault. they moved the show to friday nights at 8 : 00 p. m., then known as the " family hour ". never should have been on fridays in the first place. i was a sophomore in high school and loved the early episodes! it was first up against police woman on nbc. abc had huge problems with friday nights. bad season for them overall until barney miller, baretta, and swat debuted in january of ' 75. kolchak should have been a hit. darren mcgavin begged to get out of his contract to end the show. too bad the writing wasn ' t up to richard matheson ' s in the original tv movies. still, mcgavin made kolchak his own, as actors can do. jackie gleason as ralph kramden and caroll o ' connor as archie bunker come to mind. that ins set with the manual typewriters and clacking teletypes seems quaint and ancient today, yet that was part of the appeal. they were very lucky to have simon oa

In [ ]:
save_dir = Path("../../data/tokenized")
save_dir.mkdir(parents=True, exist_ok=True)

torch.save(tokens_train, save_dir / "imdb_train_imdb_tok.pt")
torch.save(tokens_val, save_dir / "imdb_val_imdb_tok.pt")
torch.save(tokens_test, save_dir / "imdb_test_imdb_tok.pt")